In [1]:
import json
import requests
import pandas as pd
from datetime import date, timedelta
import collections
from nltk import tokenize
from nltk.tokenize import RegexpTokenizer
from nltk import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import pickle
import time
import pytz
import datetime
import torch
import sklearn.feature_extraction
import sklearn.model_selection
import random
from sklearn.metrics import accuracy_score 
from sklearn import svm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import pynytimes
from pynytimes import NYTAPI

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [27]:
# Using New York Times API (extracting data from 2016-2020(april))

In [4]:
# News from the business section 
start_date = date(2023, 3, 31)
end_date = date(2023, 4, 28) 
delta = end_date - start_date

In [5]:
# do not run
# Getting the articles from NYT
# Using New York Times as a news Source
apikey = "#InsetAPIKey"
nytapi = pynytimes.NYTAPI(apikey, parse_dates=True)

# searching for the articles (API T&S: 6 seconds between calls)
# loop through the days getting '100' articles per day
data = []
for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    atricles_day = nytapi.article_search(
        # Query
        query = '',
        # Number of results
        results = 100,
        # Dates of articles
        dates = {
            "begin": day,
            "end": day
        },
        # Other options
        options = {
            "sort": "relevance",

            "type_of_material": [
                "News Analysis", "News", "Article"
            ] ,

            "news_desk": [
                "Financial",
                "Business",
                "Business Day",
            ],
        }
    )
    
    data += atricles_day
    
    time.sleep(6)

C:\Users\andre\anaconda3\lib\site-packages\pynytimes\helpers\article_search.py:19: UserWarning: Asking for a lot of results, because of rate limits it can take a while.
  warnings.warn(


In [6]:
news_df = pd.DataFrame(columns = ['Date', 'Headline', 'Abstract', 'Source'])
for article in data:
    news_df = news_df.append({'Date' : article['pub_date'], 'Headline' : article['headline']['main'], 'Abstract': article['abstract'], 'Source' : article['source']}, ignore_index = True)

C:\Users\andre\AppData\Local\Temp\ipykernel_9516\2274306085.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append({'Date' : article['pub_date'], 'Headline' : article['headline']['main'], 'Abstract': article['abstract'], 'Source' : article['source']}, ignore_index = True)
C:\Users\andre\AppData\Local\Temp\ipykernel_9516\2274306085.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append({'Date' : article['pub_date'], 'Headline' : article['headline']['main'], 'Abstract': article['abstract'], 'Source' : article['source']}, ignore_index = True)
C:\Users\andre\AppData\Local\Temp\ipykernel_9516\2274306085.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append({'Date

In [7]:
news_df

,Date,Headline,Abstract,Source
0,2023-03-31 22:49:08+00:00,Tesla and Musk Lose Ruling on Factory Union Is...,A court upheld a finding that Tesla wrongly fi...,The New York Times
1,2023-03-31 20:57:22+00:00,Fox News Suffers Major Setback in Defamation Case,"A judge said the suit would go to trial, for a...",The New York Times
2,2023-03-31 20:48:38+00:00,Welcome Back to The Trump Show,A historic indictment has returned the former ...,The New York Times
3,2023-03-31 18:50:12+00:00,"Why Schwab, a Financial Giant, Got Hurt in the...","Like Silicon Valley Bank, the company holds bi...",The New York Times
4,2023-03-31 18:12:30+00:00,How a Trump-Era Rollback Mattered for Silicon ...,An under-the-radar change to the way regional ...,The New York Times
...,...,...,...,...
265,2023-04-28 10:00:31+00:00,The Eurozone Economy Shows Signs of Modest Growth,The countries that use the euro recorded econo...,The New York Times
266,2023-04-28 09:00:36+00:00,"Hollywood, Both Frantic and Calm, Braces for W...",Studios have moved up deadlines for TV writers...,The New York Times
267,2023-04-28 09:00:34+00:00,"Higher Food Prices Bring Bigger Profits, but C...",Some of the biggest packaged food companies ra...,The New York Times
268,2023-04-28 09:00:23+00:00,A ‘Rocky and Bumpy’ Economy Where Wages Are Up...,Key pay and inflation gauges have stayed stubb...,The New York Times
